<a href="https://colab.research.google.com/github/ReynardGhazyTsaqif/DIF62130_B_25_2311532014/blob/main/Praktikum3/NaiveBayesModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
import pandas as pd
from sklearn.naive_bayes import BernoulliNB
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

dataset_dict = {
    'Outlook': ['sunny', 'sunny', 'overcast', 'rainy', 'rainy', 'rainy', 'overcast', 'sunny', 'sunny', 'rainy', 'sunny', 'overcast', 'overcast', 'rainy', 'sunny', 'overcast', 'rainy', 'sunny', 'sunny', 'rainy', 'overcast', 'rainy', 'sunny', 'overcast', 'sunny', 'overcast', 'rainy', 'overcast'],
    'Temperature': [85.0, 80.0, 83.0, 70.0, 68.0, 65.0, 64.0, 72.0, 69.0, 75.0, 75.0, 72.0, 81.0, 71.0, 81.0, 74.0, 76.0, 78.0, 82.0, 67.0, 85.0, 73.0, 88.0, 77.0, 79.0, 80.0, 66.0, 84.0],
    'Humidity': [85.0, 90.0, 78.0, 96.0, 80.0, 70.0, 65.0, 95.0, 70.0, 80.0, 70.0, 90.0, 75.0, 80.0, 88.0, 92.0, 85.0, 75.0, 92.0, 90.0, 85.0, 88.0, 65.0, 70.0, 60.0, 95.0, 70.0, 78.0],
    'Wind': [False, True, False, False, False, True, True, False, False, False, True, True, False, True, True, False, False, True, False, True, True, False, True, False, False, True, False, False],
    'Play': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes']
}

df = pd.DataFrame(dataset_dict)

df

,Outlook,Temperature,Humidity,Wind,Play
0,sunny,85.0,85.0,False,No
1,sunny,80.0,90.0,True,No
2,overcast,83.0,78.0,False,Yes
3,rainy,70.0,96.0,False,Yes
4,rainy,68.0,80.0,False,Yes
5,rainy,65.0,70.0,True,No
6,overcast,64.0,65.0,True,Yes
7,sunny,72.0,95.0,False,No
8,sunny,69.0,70.0,False,Yes
9,rainy,75.0,80.0,False,Yes


Import library yang dibutuhkan dan generate dummy dataset secara manual

In [59]:
df = pd.get_dummies(df, columns=['Outlook'], prefix='', prefix_sep='', dtype=int)

df['Wind'] = df['Wind'].astype(int)

df['Play'] = (df['Play'] == 'Yes').astype(int)

df.head()

,Temperature,Humidity,Wind,Play,overcast,rainy,sunny
0,85.0,85.0,0,0,0,0,1
1,80.0,90.0,1,0,0,0,1
2,83.0,78.0,0,1,1,0,0
3,70.0,96.0,0,1,0,1,0
4,68.0,80.0,0,1,0,1,0


2. Data preprocessing
1. Konversi fiutr outlook degngan  algroitma OHE yg ubah nilai kategori ke biner. lalu fitur wind dan kelas/label play juga konversi ke biner

lalu temperatur <= 80 dan 80, warm, hot
humidity <=75 dan >75, dry, humid

In [60]:

df['Temperature'] = pd.cut(df['Temperature'], bins=[0, 80, 100], labels=['Warm', 'Hot'])

df['Humidity'] = pd.cut(df['Humidity'], bins=[0, 75, 100], labels=['Dry', 'Humid'])

one_hot_columns = pd.get_dummies(df[['Temperature', 'Humidity']],drop_first=True, dtype=int)

df = df.drop(['Temperature', 'Humidity'], axis=1)

df = pd.concat([one_hot_columns, df], axis=1)

df.head()

,Temperature_Hot,Humidity_Humid,Wind,Play,overcast,rainy,sunny
0,1,1,0,0,0,0,1
1,0,1,1,0,0,0,1
2,1,1,0,1,1,0,0
3,0,1,0,1,0,1,0
4,0,1,0,1,0,1,0


In [61]:
X, y = df.drop(columns='Play'), df['Play']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=False)

bagi data menjadi data latih dan data uji

Main Mechanism

-Hitung probabilitas tia kelas dalam data pealtihan

-tiap fitur dan kelas, hitung probabilitas fitur bernilai 0 da 1

-utk data baru: tiap kelas, kalikan probabilitasnya dengan probabilitas tiap nilai fitur 0/1

-prediksi kelas dengan probabilitas tertinggi





In [62]:
from fractions import Fraction

def calc_target_prob(attr):
  total_counts = attr.value_counts().sum()
  prob_series = attr.value_counts().apply(lambda x: Fraction(x, total_counts).limit_denominator())
  return prob_series

print(calc_target_prob(y_train))




Play
1    11/19
0     8/19
Name: count, dtype: object


2. feature probability calculation: utk tiap fitur dan kelas hitung
jumlah kejadian dimana fitur 0 dieklas ini
jumlah kejaidan dimana fitur 1 dikelas iin

In [63]:
from fractions import Fraction

def sort_attr_label(attr, lbl):
    return (pd.concat([attr, lbl], axis=1)
            .sort_values([attr.name, lbl.name])
            .reset_index()
            .rename(columns={'index': 'ID'})
            .set_index('ID'))

def calc_feature_prob(attr, lbl):
    total_classes = lbl.value_counts()
    counts = pd.crosstab(attr, lbl)
    prob_df = counts.apply(lambda x: [Fraction(c, total_classes[x.name]).limit_denominator() for c in x], axis=0)
    return pd.DataFrame(prob_df)

print(sort_attr_label(X_train['sunny'], y_train))
print(calc_feature_prob(X_train['sunny'], y_train))

    sunny  Play
ID             
5       0     0
13      0     0
2       0     1
3       0     1
4       0     1
6       0     1
9       0     1
11      0     1
12      0     1
15      0     1
16      0     1
0       1     0
1       1     0
7       1     0
14      1     0
17      1     0
18      1     0
8       1     1
10      1     1
Play     0     1
sunny           
0      1/4  9/11
1      3/4  2/11


In [64]:
for col in X_train.columns:
  print(calc_feature_prob(X_train[col], y_train), "\n")

Play               0     1
Temperature_Hot           
0                5/8  9/11
1                3/8  2/11 

Play              0     1
Humidity_Humid           
0               1/4  4/11
1               3/4  7/11 

Play    0     1
Wind           
0     3/8  8/11
1     5/8  3/11 

Play      0     1
overcast         
0         1  6/11
1         0  5/11 

Play     0     1
rainy           
0      3/4  7/11
1      1/4  4/11 

Play     0     1
sunny           
0      1/4  9/11
1      3/4  2/11 



In [65]:
nb_clf = BernoulliNB()
nb_clf.fit(X_train, y_train)

BernoulliNB()

In [66]:
y_pred = nb_clf.predict(X_test)

print(accuracy_score(y_test, y_pred))

0.5555555555555556
